# Compiler

## Network container class

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

class Program:
    # this class contains in a structured manner matrices and biases
    # which encode the semantics of the program network, in the same fashion
    # of the source code of a program
    def __init__(self, topology , activation_functions_list):
        self.topology = topology
        self.activation_functions_list = activation_functions_list
        self.W = [ 0 for t in topology[:-1]]
        self.b = [ 0 for t in topology[:-1]]
    def random_init_weights(self):
        for layer in range(len(self.topology[:-1])):
            self.W[layer] = np.random.normal( size = (self.topology[layer + 1] , self.topology[layer] ) )**2
            size = self.W[layer].shape[0] * self.W[layer].shape[1] 
            for i in range(size - int(np.sqrt(size))):
                r = np.random.choice(len(self.W[layer]))
                c = np.random.choice(len(self.W[layer].T))
                self.W[layer][r,c] = 0.   # adding some sparsity
                
            self.b[layer] = np.random.normal( size = self.topology[layer + 1] )
    def print(self):
        for i,layer in enumerate(self.topology[:-1]):
            print("layer %d->%d "%(i,i+1))
            print("W.shape = %s \t b.shape = %s" % (str(self.W[i].shape),str(self.b[i].shape)))
    def visualize_weights(self):
        for w in self.W:
            print(w.shape)
            plt.figure(figsize=(10,10))
            plt.imshow(w > 0.)        

## Sparsification algorithm

## IR production

### Lightweight trees representation

In [3]:
class c:
    # every element is a node in the tree.
    # the first argument denotes the name of the node ; the possible next arguments are a list of the sons.
    # Note that nodes without sons are simply leaf (eg. ADD, MUL,...)
    def __init__(self, ID, *args):
        self.id   = ID
        self.sons = list()
        for arg in args:
            self.sons.append(arg)
    def print(self,level = 0):
        print( ("\t" * level) + str(self.id) )
        for s in self.sons:
            s.print(level + 1)
    def __str__(self):
        ret = str(self.id)
        if len(self.sons) > 0:
            ret += "("
            for s in self.sons:
                ret += str(s)
                if s != self.sons[-1]:
                    ret += ','
            ret += ")"
        return ret
    def flatten(self):
        ret = []
        ret += [self.id]
        for s in self.sons:
            ret += s.flatten()
        return ret

### IR production function

In [4]:
def IR(program, compile_time_data = True):
    # for every layer
    ### print("y = input")
    ret = list()
    
    for layer,t in enumerate(program.topology[1:]):
        IR_instruction = c("COMMENT", c("START"))
        ### print(IR_instruction)
        ret.append(IR_instruction)
        
        f = program.activation_functions_list[layer]
        R_offset = t
        for i in range(len(program.b[layer])): # for every row
            #print("R[%d] = 0" % (i))
            #print("MOVE( TEMP(%d) , CONST(0) )" % (i + R_offset))
            IR_instruction = c(
                    "MOVE",
                     c("TEMP" , c(i + R_offset) ),
                     c("CONST", c(0))
            )
            ### print(str(IR_instruction))
            ret.append(IR_instruction)
            
            
        for i in range(len(program.W[layer])): # for every row
            for j in range(len(program.W[layer].T)): # for every column
                if( program.W[layer][i,j] != 0):
                    # if there is zero is useless to compute the contribution
                    #print("->MOVE(TEMP(%d),BINOP(ADD,TEMP(%d),BINOP(MUL,CONST(%f),TEMP(%d)))) " % (i + R_offset,i + R_offset, program.W[layer][i,j],i )) # COMPILE DATA HYPOTHESIS
                    IR_instruction = c(
                        "MOVE",
                        c("TEMP",
                            c(i + R_offset)
                        ),
                        c("BINOP",
                            c("ADD"),
                            c("TEMP",
                                c(i + R_offset)
                            ),
                            c("BINOP",
                                c("MUL"),
                                c("CONST",
                                     c(program.W[layer][i,j])
                                 ),
                                c("TEMP",
                                 c(j)
                                 )
                            )
                        )
                    )
                    #print(IR_instruction)
                    ret.append(IR_instruction)
                else:
                    0
                    ### print("# here there was a 0 so we exploit sparsity ")
                    
        for i in range(len(program.b[layer])): # for every row
            # a priori in compile time since the amount of "repetitions" doens't scale quadratically , in opposite to weights
            #print("->MOVE(TEMP(%d),BINOP(ADD,TEMP(%d),CONST(%f)))" % (i, i + R_offset , program.b[layer][i]) )
            IR_instruction = c("MOVE",
                                c("TEMP",
                                 c(i)
                                 ),
                                c("BINOP",
                                 c("ADD"),
                                 c("TEMP",
                                   c(i + R_offset)
                                  ),
                                 c("CONST",
                                  c(program.b[layer][i])
                                  )
                                 )
                                )
            ### print((IR_instruction))
            ret.append(IR_instruction)
        for i in range(len(program.b[layer])): # for every row
            #print("y[%d] = f(y[%d])" % (i,i))
            #print("MOVE( TEMP(%d) , CALL( %s, TEMP(%d) ) )" % (i,f,i) )
            IR_instruction = c("MOVE",
                c("TEMP",
                     c(i)
                 ),
                c("CALL",
                     c(f),
                     c("TEMP",
                          c(i)
                      )
                 )
            )
            ### print(IR_instruction)
            ret.append(IR_instruction)
        IR_instruction = c("COMMENT", c("END"))
        ### print(IR_instruction)
        ret.append(IR_instruction)
    return ret


## Register and memory allocation

### Temporary variables statistics class

In [5]:
class TemporaryVariablesStatistics:
    def __init__(self):
        self.temp_usage_map = {}
    def increment(self,temp_variable):
        old_value = self.temp_usage_map.get(temp_variable)
        if old_value == None:
            old_value = 0
        self.temp_usage_map[temp_variable] = old_value + 1
    def get_data(self):
        return self.temp_usage_map
    def vectorize(self):
        arr = []
        for s in self.get_data():
            arr.append( [ s, self.get_data()[s]] )
        arr = np.array(arr)                                                              # builds a tempstable [ temp | usage ]
        arr = arr[ arr[:,1].argsort()[-1::-1] ]                                          # sort the tempstable by usage  (decreasing)
        return arr


### Register allocation data class

In [6]:
class RegisterAllocationData:
    def __init__(self):
        self.temp_reg_map = {}
    def insert(self,temp_variable, register):
        self.temp_reg_map[temp_variable] = register
    def get_data(self):
        return self.temp_reg_map
    def print(self):
        for t in self.temp_reg_map:
            print(t , "\t", self.temp_reg_map[t] )
    def contains(self,tmp_name):
        return tmp_name in self.temp_reg_map

### Memory allocation data class

In [77]:
class MemoryAllocationData:
    def __init__(self):
        self.temp_mem_map = {}
        
    def insert(self,temp_variable, address):
        self.temp_mem_map[temp_variable] = address
        
        
    def batch_set(self, list_of_temps, list_of_addresses):
        for tmp_id, mem_addr in zip(list_of_temps,list_of_addresses):
            self.temp_mem_map[tmp_id] = mem_addr
        
    def get_data(self):
        return self.temp_mem_map
    def get_variables_list(self):
        ret = list()
        for s in self.temp_mem_map:
            ret.append(s)
        return ret
    
    def get_input_temps(self, prev_layer_size):
        all_vars = np.array(self.get_variables_list())
        return all_vars[all_vars < prev_layer_size]
    
    def get_output_temps(self,prev_layer_size):
        all_vars = np.array(self.get_variables_list())
        return all_vars[all_vars >= prev_layer_size]
    
    def print(self):
        for t in self.temp_mem_map:
            print(t , "\t", self.temp_mem_map[t] )

### Block Signals

In [78]:
class BlockSignals:
    def __init__(self, memory_allocation_object):
        # initialize an empty dictionary starting from the variables name
        self.memory_allocation_object = memory_allocation_object
        self.temp_signals_map = {}
        for t in memory_allocation_object.get_data():
            self.temp_signals_map[t] = []
        
    def add_tick(self, temp_variables):
        temp_variables          = np.intersect1d(temp_variables, self.memory_allocation_object.get_variables_list())
        all_temporary_variables = self.memory_allocation_object.get_variables_list()
        # push 0 in the lists of unused temps and 1 in the list of the used temp
        for t in all_temporary_variables:
            self.temp_signals_map[t].append(0)
        for t in temp_variables:
            self.temp_signals_map[t][-1] = 1.
            
    def get_data(self):
        return self.temp_signals_map

### Embedding

### Theoretical argument : memory allocation

Suppose that we have some kind of distance between temporary variables in a <b>matrixmult</b>. <br>
Therefore we have a collection of distance matrices $\{D_{1,2},D_{2,3},...,D_{N-1,N}\}$ <br>
We can formulate the following optimization problem <br><br>
$
    \text{Find $\{P_{1,2},P_{2,3},...,P_{N-1,N}\}$ permutations of 
    $\{ [1,n_{1,2}], [1,n_{2,3}],... ,[1,n_{N-1,N}] \}$ }
$ <br>
$
\text{such that
    $D(P_{i,i+1}) \sim D_{i,i+1} \ \ \ \forall i$
}
$ <br>
$
\text{Subject to
    $ (P_{i,i+1})_{\text{output (only memory)}} = (P_{i+1,i+2})_{\text{input (only memory)}}  \ \ \forall i$
}
$

### Proposed algorithm

$
\text{$P_0 \leftarrow$ Permutation$(Temp_{\mathcal l,\mathcal l + 1})$ optimal}\\
\text{${\bf for } \ \ \ i,(layer,layer+1) \in temps $ : } \\
\hspace{2em} \text{$ P_i \leftarrow $ Permutation$(Temp_{\mathcal l_i,\mathcal l_{i+1}})$ optimal subject to $P_i^{input} = P_{i-1}^{output}$ }
$

### Constrained Permutation Class

In [79]:
class constrainedPermutation:
    def __init__(self, constraint, trivial_init = True, init_vector = None):
        # constraint = vector of -1, with the exception of the "fixed points" of the permutations
        #              e.g.   -1 -1 4 3 -1
        #              reads as "all the permutations of 01234" such that the third element is 4 and the fourth is 3
        self.constraint = constraint
        if trivial_init == True:
            self.value      = np.arange(len(constraint))
            for i in np.arange(len(constraint))[constraint != -1]:
                # find the value at the position i
                to_swap_A = i                                      # position 1
                to_swap_B = np.argmax(self.value == constraint[i]) # position 2
                tmp       = self.value[to_swap_A]
                self.value[to_swap_A] = self.value[to_swap_B]
                self.value[to_swap_B] = tmp
        else:
            self.value = init_vector.copy()
    
    def get_neighbor(self):
        feasible = np.arange(len(self.constraint))[self.constraint == -1]
        old = self.value[feasible]
        a        = np.random.choice(len(feasible))
        b        = np.random.choice(len(feasible))
        temp     = feasible[a]
        feasible[a] = feasible[b]
        feasible[b] = temp
        ret = self.value.copy()
        ret[feasible] = old
        return constrainedPermutation(self.constraint,False,ret)
    
    def get_indexes(self):
        return self.value
    
    def distance_matrix(self):
        return np.array([
            [
                    np.linalg.norm(a - b)
                for b in self.value
            ]
            for a in self.value
        ])
    
    def print(self):
        print(self.value)
ret = list()
for i in range(10000):
    ret.append( constrainedPermutation(np.array([-1.,-1.,2,3,4,-1,-1,-1])).get_neighbor().value )
ret = np.array(ret)
ret.mean(axis = 0)

array([1.5064, 2.0995, 2.    , 3.    , 4.    , 4.5186, 5.1512, 5.7243])

In [80]:
def affinity(permutation, distance_matrix):
    # compute the permutation distance matrix
    d_hat = permutation.distance_matrix()
    return np.linalg.norm( d_hat - distance_matrix.argsort(axis = 1) )

### Allocator class

In [85]:
from sklearn.manifold import MDS
class Allocator:
    def __init__(self, ir, program, register_names):
        self.register_allocation_data = []  # register allocations for every matrix multiplication operation
        self.memory_allocation_data   = []  #   memory allocations for every matrix multiplication operation
        self.program                  = program
        # ------------------------------
        # Register allocation subroutine
        temp_statistics = self.most_used_temps(ir)   # produces a list of register statistics objects
        self.register_allocation_and_memory_alloc_init(temp_statistics, register_names)
        self.memory_allocation(
                self.compute_signals_distance_matrix(
                    self.compute_signals(ir)
                )
        )
    ########################################################################################
    # Register allocation
    ########################################################################################
    
    def most_used_temps(self, ir):
        # IN   : takes as input an intermediate representation
        # OUT  : produces a list of TemporaryVariableStatistics objects, one for each matrix mult
        statistics_per_block = list()
        
        for ir_instruction in ir:                                                            # iterate over the IR statements
            if(ir_instruction.id == "COMMENT"):                                              # 
                if(ir_instruction.sons[0].id == "START"):
                    statistics_per_block.append(TemporaryVariablesStatistics())              # i create a temporaryvariablestatistcs
            else:
                unrolled_ir = ir_instruction.flatten()                                       # unroll the statemenet
                temps_in_statement   = list()                                                # container for temps variables in the current statement
                for u,val in zip(unrolled_ir[:-1],unrolled_ir[1:]):  
                    if u == "TEMP": 
                        temps_in_statement.append(val)                                       
                                                                                             # now "temps_in_statement" contains only the values of the temporary variables
                for t in temps_in_statement:                                                 # count the usage of each temporal 
                    statistics_per_block[-1].increment(t)                                       # the current temporary variable statistics is updated 
        return statistics_per_block
    
    def register_allocation_and_memory_alloc_init(self, statistics_list , register_names):
        # IN  : a statistics list obtained from  most_used_temps , register names
        # OUT : a registerAllocation object
        
        # convert the dictionary to an array
        temp_stats_per_block = list()
        for stat in statistics_list:
            # I transform the dictitonary in a sorted-by-usage vector
            arr = stat.vectorize()
            
            # I initialize a Registerallocation  object
            reg_data = RegisterAllocationData()   
            
            # for every register i take an element, starting from the beginning, of the array
            temp_var_count = 0
            for r in register_names:
                if temp_var_count >= len(arr):
                    break
                reg_data.insert(arr[temp_var_count,0],r)                                      # i add as a used register the temporary variables with more usage
                temp_var_count += 1
            self.register_allocation_data.append(reg_data)                                    # i append the register allocation data obtained to the list of RAD
            
            # I also initialize the "slots" for the memory allocation data
            mem_data = MemoryAllocationData()                                                 
            for t in range(temp_var_count, len(arr)):
                mem_data.insert(arr[t,0], -1)                                                 # initializa with -1
            self.memory_allocation_data.append(mem_data)                                      # i add them to the list
        
        return 0
    
    ########################################################################################
    # Memory allocation
    ########################################################################################

    def compute_signals(self, ir):
        # takes as input an intermediate repr and a register allocation output
        signals_per_block = list()
 
        curr_alloc_block = 0
    
        for ir_instruction in ir:
            if(ir_instruction.id == "COMMENT"):
                if(ir_instruction.sons[0].id == "START"):
                    curr_memory_alloc_block = self.memory_allocation_data[curr_alloc_block]
                    signals_per_block.append(BlockSignals(curr_memory_alloc_block))                       
                    curr_alloc_block += 1

            unrolled_ir = ir_instruction.flatten()
            temps_in_statement   = list()
            for u,val in zip(unrolled_ir[:-1],unrolled_ir[1:]):
                if u == "TEMP":
                    temps_in_statement.append(val)      
                    
            signals_per_block[-1].add_tick(temps_in_statement)
        return signals_per_block
    
    def compute_signals_distance_matrix(self, signals):
        # IN   : takes as input a collection of TempVarSignals
        # OUT  : produces a 
        
        mappings = list()          # mapping between the rows of the matrix and the temp_var
        Ds = list()                # list of matrices
        
        for signals_block in signals:
            D = np.zeros((len(signals_block.get_data()),len(signals_block.get_data())))
            mapping = {}
            for i,a in enumerate(signals_block.get_data()):
                mapping[i] = a
                for j,b in enumerate(signals_block.get_data()):  
                    v_a = np.arange(len(signals_block.get_data()[a]))[np.array(signals_block.get_data()[a]) == 1.]
                    v_b = np.arange(len(signals_block.get_data()[b]))[np.array(signals_block.get_data()[b]) == 1.]
                    distanza = 0.5 * (np.mean([ np.min(np.abs(s_1 - v_b)) for s_1 in v_a]) + np.mean([ np.min(np.abs(s_2 - v_a)) for s_2 in v_b]))
                    D[i,j] = distanza
            Ds.append(D)
            mappings.append(mapping)
        return Ds, mappings
    
    def anneal(self,constrainedperm, Ds, mapping):
        # stupid stub function
        ret = constrainedPermutation(
            constrainedperm.constraint,
            False,
            constrainedperm.value
        )
        
        for i in range(100):
            ret = ret.get_neighbor()
            
        return ret
    
    
    
    def density_optimizer_memory_subset_for_output(self,memory,constraint_vector, input_size, output_size):
        memory_mask = np.arange(len(memory))[ [ not( x in constraint_vector) for x in np.arange(len(memory))] ]
         
        rows = list()
        densities = list()
        
        for j in range( (len(memory) - input_size) - output_size + 1):
            row = np.zeros(len(memory))
            
            row[ constraint_vector[constraint_vector != -1].astype(int)] = -1.
            row[memory_mask[j:j+output_size]] = 1.
            density = lambda r : (r != 0 )[ (r != 0).argmax() : (len(r) - (r != 0)[-1::-1].argmax())].mean()

            rows.append(row)
            densities.append(density(row))
            
        densities = np.array(densities)
        rows      = np.array(rows)
        
        # choose the best
        configuration = rows[densities.argmax()]
        
        print(configuration)
        
        # set the output starting from the constraint vector
        perm_vector = constraint_vector.copy()
        perm_vector[np.arange(len(perm_vector))[perm_vector == -1]] = np.arange(len(memory))[configuration == 1]
        
        return perm_vector

    def memory_allocation(self, DS_MAPPINGS):
        Ds       = DS_MAPPINGS[0]
        mappings = DS_MAPPINGS[1]
        
        # i define a fake memory just for debug
        memory = np.arange(
            np.max(
                [
                        np.max(m.get_variables_list())
                    for m in self.memory_allocation_data 
                ]
            )
        )
        
        # D, mapping
        
        D       = Ds[0]
        mapping = mappings[0]
        
        # load the temps in the first matrix mult
        temps_in_mem = self.memory_allocation_data[0].get_variables_list()
        # define a random permutation on the first [0 , | temps_in_mem | - 1]
        permutation = constrainedPermutation(np.ones(len(temps_in_mem))  * -1)
        permutation = self.anneal(permutation, D, mapping)
        
        print(self.program.topology)
        temps_in_mem_input  =  self.memory_allocation_data[0].get_input_temps(self.program.topology[0])
        temps_in_mem_output =  self.memory_allocation_data[0].get_output_temps(self.program.topology[0])
        mems_input          =  permutation.value[ : len(temps_in_mem_input)]
        mems_output         =  permutation.value[ len(temps_in_mem_input) :]
        
        
        print(self.memory_allocation_data[0].get_variables_list())

        
        
        #save the permutation in the memory_allocation_data list
        self.memory_allocation_data[0].batch_set(temps_in_mem, permutation.value)
        
        
        for i in range(1, len(self.program.topology) - 1):
            
            temps_in_mem_input  =  self.memory_allocation_data[i].get_input_temps(self.program.topology[i])
            mems_input          =  mems_output.copy()
            temps_in_mem_output =  self.memory_allocation_data[i].get_output_temps(self.program.topology[i])
            mems_output         =  np.ones(len(temps_in_mem_output)) *-1
            
            
            print(len(temps_in_mem_input),len(temps_in_mem_output))
            
            # define the constraint vector
            constraint_vector   = np.r_[mems_input, mems_output]
            
            # compute the density-wise optimal position of the mapping of the whole output subset of temps
            permutation_vector  = self.density_optimizer_memory_subset_for_output(memory,
                                                                             constraint_vector,
                                                                             len(temps_in_mem_input),
                                                                             len(temps_in_mem_output)
                                                                            )
            

        

## Code generator

## Assembler

## Orchestrator function

In [89]:
def compiler(file_name, registers, sparsify = False):
    # debug, we don't actually read a file but we generate it randomly
    rete = Program([5,30,30,1], ["RELU","RELU","LINEAR"])
    rete.random_init_weights()
    intermediate_representation = IR(rete)
    allocator = Allocator(intermediate_representation , rete , registers[2:])
    asm_code = []
    
    #for m in allocator.memory_allocation_data:
    #    m.print()
    #    print("----")
    
    cursor_allocator = 0
   
    for ir_statement in intermediate_representation:
        flattened = ir_statement.flatten()
        #print(flattened)
        if(flattened[0] == "COMMENT"):
            if(flattened[1] == "END"):
                #print("\n # NUOVO BLOCCO \n")
                cursor_allocator += 1
        
        if(flattened[0] == "MOVE" 
           and 
           flattened[3] == "CONST"
          ): # set the temporary variable
            tmp_name = flattened[2]
            val       = flattened[4]
            # we have to understand if tmp_name is a register or not
            if(allocator.register_allocation_data[cursor_allocator].contains(tmp_name)):
                0
                #print("MOV TO $%d THE VALUE #%f" % (
                #      allocator.register_allocation_data[cursor_allocator].get_data()[tmp_name],
                #      val)
                #     )
            else:
                # variable is in memory
                address = allocator.memory_allocation_data[cursor_allocator].get_data()[tmp_name]
                #print("STORE THE VALUE 0 IN ADDRESS %d" % address)
                
                
        if(flattened[0] == "MOVE"
           and
           flattened[3] == "BINOP"
           and 
           len(flattened ) == 13
          ): # addition and multiply
            neuron_dest   = flattened[2]
            neuron_source = flattened[12]
            weight        = flattened[10]
            
            
            if(allocator.register_allocation_data[cursor_allocator].contains(neuron_source)):
                0
                #print("MULT $%d BY #%f AND SAVE IT IN $0" % (
                #      allocator.register_allocation_data[cursor_allocator].get_data()[neuron_source],
                #      weight)
                #     )
            else:
                # variable is in memory
                address = allocator.memory_allocation_data[cursor_allocator].get_data()[neuron_source]
                #print("LOAD IN REGISTER $0 THE ADDRESS %d"  % allocator.memory_allocation_data[cursor_allocator].get_data()[neuron_source])
                #print("MULT $0 BY #%f AND SAVE IT IN $0" % (
                #      weight)
                #     )
            if(allocator.register_allocation_data[cursor_allocator].contains(neuron_dest)):
                0
                #print("ADD $0 to $%d" % (
                #      allocator.register_allocation_data[cursor_allocator].get_data()[neuron_dest])
                #     )
            else:
                # variable is in memory
                address = allocator.memory_allocation_data[cursor_allocator].get_data()[neuron_dest]
                #print("LOAD IN REGISTER $1 THE ADDRESS %d" % address)                
                #print("ADD $0 to $1")
                #print("STORE $1 IN ADDRESS %d" % address)
    
compiler(0, np.arange(3))

[5, 30, 30, 1]
[4, 0, 3, 2, 40, 52, 43, 42, 53, 59, 46, 34, 33, 31, 55, 51, 45, 57, 44, 41, 38, 37, 36, 35, 32, 58, 48, 50, 54, 39, 56, 30, 28, 16, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 15, 5, 14, 13, 12, 11, 10, 9, 8, 7, 6, 29, 47, 49]
30 29
[-1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1.  1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.]


ValueError: shape mismatch: value array of shape (4,) could not be broadcast to indexing result of shape (29,)

# Test

## Unicorn emulator

<hr>

In [72]:
1 in {1:3,5:5}

True